In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# CHECK WORKIND DIR PATH
import os
# MODIFY SYSTEM PATH
os.chdir('/Users/igorgarbuz/Dev/catlearn')
os.getcwd()

In [ ]:
from typing import Callable, Iterable, Any
import sys
import torch
import random
import warnings
# Needed to show warnings in all Jupyter distributions (e.g. VS Code Jupyter implementation)
warnings.simplefilter(action="default")
from tqdm import tnrange
sys.path.append('../catlearn')
from catlearn.data.dataset import Dataset
from catlearn.tensor_utils import (Tsor, DEFAULT_EPSILON)
from catlearn.graph_utils import (DirectedGraph, uniform_sample, random_walk_edge_sample,
                                    random_walk_vertex_sample)
from catlearn.algebra_models import (Algebra, VectAlgebra, VectMultAlgebra)
from catlearn.composition_graph import CompositeArrow
from catlearn.categorical_model import (TrainableDecisionCatModel, RelationModel,
                                        ScoringModel)

In [ ]:
# DEBUG: Tools to export networkx graph
from networkx import to_dict_of_lists, to_dict_of_dicts

In [ ]:
# Select default type: https://pytorch.org/docs/stable/tensors.html
default_tensor = torch.FloatTensor
# default_tensor = torch.cuda.FloatTensor
torch.set_default_tensor_type(default_tensor)
# Check default tensor datatype
default_tensor.dtype

In [ ]:
class CustomRelation(RelationModel):
    """ Fake relations
    two nodes and rel
    """
    def __init__(self, nb_features: int, nb_labels: int, algebra: Algebra) -> None:
        self.linear = torch.nn.Linear(2 * nb_features + nb_labels, algebra.flatdim)

    @property
    def parameters(self) -> Callable[[], Iterable[Any]]:
        return self.linear.parameters

    def __call__(self, x: Tsor, y: Tsor, l: Tsor) -> Tsor:
        """ Compute x R y """
        return self.linear(torch.cat((x, y, l), -1))

In [ ]:
class CustomScore(ScoringModel):
    """ Must be defined. depends on algebra and 
    in the scope of definition of the project. """
    def __init__(
            self,
            nb_features: int,
            nb_scores: int,
            algebra: Algebra) -> None:
        self.linear = torch.nn.Linear(
            2 * nb_features + algebra.flatdim, nb_scores + 1)
        self.softmax = torch.nn.Softmax(dim=-1)

    @property
    def parameters(self) -> Callable[[], Iterable[Any]]:
        return self.linear.parameters

    def __call__(self, src: Tsor, dst: Tsor, rel: Tsor) -> Tsor:
        """ Compute S(src, dst, rel) """
        cat_input = torch.cat((src, dst, rel), -1)
        return self.softmax(self.linear(cat_input))[..., :-1]

In [ ]:
# Specify dataset path
ds_path_wn18 = '/Users/igorgarbuz/Dev/catlearn/datasets/wn18rr/text'
ds_path_fb15 = '/Users/igorgarbuz/Dev/catlearn/datasets/fb15k-237'

In [ ]:
ds_wn18 = Dataset(path=ds_path_wn18, ds_name='wn18', node_vec_dim=10)

In [ ]:
# ds_fb15 = Dataset(path=ds_path_fb15, ds_name='fb15', node_vec_dim=10)

In [ ]:
# MODIFY BELOW TO USE RIGHT DATASET
ds = ds_wn18

In [ ]:
# DEBUG: convert dataset to a list
# ds_l = list(ds.train)

In [ ]:
algebra = VectMultAlgebra(ds.entity_vec_dim)

In [ ]:
relation_model = CustomRelation(
    nb_features=ds.entity_vec_dim,
    nb_labels=len(ds.relation2id),
    algebra=algebra
)

In [ ]:
scoring_model = CustomScore(
    nb_features=ds.entity_vec_dim,
    nb_scores=len(ds.relation2id),
    algebra=algebra
)

In [ ]:
model = TrainableDecisionCatModel(
    relation_model=relation_model,
    label_universe=ds.relation_id2vec,
    scoring_model=scoring_model,
    algebra_model=algebra,
    optimizer=torch.optim.Adam,
    epsilon=DEFAULT_EPSILON
)

In [ ]:
# DEBUG: Represent graph as dict of dicts or dict of lists
# dod = to_dict_of_dicts(labels)
# dod[10698]
# dol = to_dict_of_lists(labels)
# dol[10698]

In [ ]:
# DEBUG: check OHE relations encoding
ds.relation_id2vec

In [ ]:
# DEBUG: check relations to id conversion dict
ds.relation2id

In [ ]:
# DEBUG NOTE: datatype comparaison
# Dataset interable format [src:int, tgt:int, lbl: {id:int: vec:Tsor}]
# CompositeArrow data format:  [[src: int, tgt: int], [label: int]]

In [ ]:
graph_labels = DirectedGraph(ds.train)

In [ ]:
def graph_to_composite_arrow(graph: DirectedGraph):
    nodes = [(src, dst) for src, dst, _ in graph.edges(data=True)]
    edges = [list(rel.keys()) for _, _, rel in graph.edges(data=True)]
    return nodes, edges

In [ ]:
# NOTE: for large graphs, random_walk functions family can be used to sub-sample graph
# while preserving its topology 
for i in tnrange(10, desc='1st loop: batches'):
    arrows_graph = uniform_sample(graph=graph_labels, sample_vertices_size=7, rng=random.Random(), with_edges=True)
    nodes, edges = graph_to_composite_arrow(arrows_graph)
    if edges:
        arrows = [CompositeArrow(nodes=node, arrows=edge) for node, edge in zip(nodes, edges)]
    else:
        arrows = [CompositeArrow(nodes=node) for node in nodes]
    _cache, _matches = model.train(
        data_points = ds.entity_id2vec,
        relations = arrows,
        # NOTE: Labels could be a couplete graph, a subgraph from random_walk or a sub-sub-graph used to create a batch
        labels = arrows_graph,
        step = True,
        match_negatives=False
    )